# LLM examples

## Learning goals: 

- Understand the concept and mechanisms of "tools" in the context of LLMs by examples. 

In this notebook, we will manually create a mechanism to call an external "tool" to complement the LLM's output. This is an exercise to understand the prompting strategy. 

(In practice, most LLMs now have systematic ways to specify new tools and each provides a specific syntax to specify tools. Also note that working with tools, and some of the related prompting strategies, are very recent and there is still active research in this). 

## Dependencies

We will use the packages `cohere` (see notebook 1) and `openfoodfacts`. Make sure these are installed. 

## Environment preparation

In [4]:
import cohere
import openfoodfacts

/Users/joseantonio.rodriguez15/Library/CloudStorage/OneDrive-UniversitatRamónLlull/projects-ES-FR62M6XQ1J/teaching_prototyping/consulta/PDAI26/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
with open("cohere.key") as f:
    COHERE_API_KEY = f.read()
cohere_client = cohere.ClientV2(COHERE_API_KEY)
api = openfoodfacts.API(user_agent="MyAwesomeApp/1.0")


## Working with tools

Imagine we ask Cohere about the calories of some food item. While it will still provide an answer, this answer will be based on the word statistics learned by the model. 

In [7]:
prompt = input("Enter some text")

response = cohere_client.chat(
    messages=[{
        "role": "user",
        "content": prompt
    }],
    model="command-a-03-2025"
)

print(response.message.content[0].text)

The calorie content of a bag of Doritos can vary depending on the size and flavor of the bag. Here are some general estimates for common sizes:

1. **Small Bag (1 oz / 28 g)**:  
   - Approximately **140–150 calories**.

2. **Regular Bag (2 oz / 56 g)**:  
   - Approximately **280–300 calories**.

3. **Family Size Bag (11.5 oz / 326 g)**:  
   - Approximately **1,400–1,500 calories**.

These estimates are for the classic Nacho Cheese flavor. Other flavors may have slightly different calorie counts. Always check the nutrition label on the specific bag for accurate information.


Let's say, in order to have a more accurate answer, we want to connect this LLM with the OpenFoodFacts database. 

First, we create a function that queries the database given the name of the product and returns the calories: 

In [8]:
def get_calories(name):
    """
    Get the calories of a product by its name.
    """
    #result = api.product.text_search(name)

    # This just takes the first item of the search. 
    # In a real application, you would need to do more error checking.
    #first_match = result['products'][0]
    #id = first_match['_id']
    #result = api.product.get(id, fields=["code", "product_name", "energy-kcal_100g"])
    # return result['energy-kcal_100g']
    return 100


In [9]:
get_calories("Oreo")

100

Now we can use the following strategy for the LLM to call this function if needed. 

- We do a first call to the LLM to check if the prompt asks about calories. If not, respond normally. 
- If the prompt asks about calories, respond with some internal "code" so that we can intercept it and do further processing. 
- If we detect the code, then we call the above function. 
- Finally, we create a new prompt  asking the LLM to provide a final response based on the original request and the information returned by the function. 

In [14]:
message = input("Enter some text")
system_prompt = """
This is a normal chatbot. Except if it the user asks for the calories of a food item,
return the message CALORIES_REQUEST followed by the product name,
For example, if the user asks for the calories of an apple, return CALORIES_REQUEST apple.
"""

response = cohere_client.chat(
    messages = [
        {'role':'system', 'content': system_prompt},
        {'role': 'user', 'content': message}
    ],
    model="command-a-03-2025"
)

output = response.message.content[0].text

if output.startswith("CALORIES_REQUEST"):
    product_name = output.split(" ")[1]
    calories = get_calories(product_name)
    print("<INTERMEDIATE RESULT> Calories=", calories)
    final_response = cohere_client.chat(
        messages=[
            {
            'role': 'system',
            'content': f"""
Now complete the request of the user, knowing 
the calories of the product on the OpenFoodFacts database is {calories}.
Mention this source in the response.
            """
            },
            {"role": "user", "content": message}
        ],
        model="command-a-03-2025")
    print("Final response: ", final_response.message.content[0].text)
else:
    print("Final response: ", output)



<INTERMEDIATE RESULT> Calories= 100
Final response:  According to the OpenFoodFacts database, the calories of one bag of Doritos is 100. Therefore, the calories of two bags of Doritos would be:

2 bags × 100 calories/bag = **200 calories**

Please note that the actual calorie content may vary depending on the specific type and size of Doritos, as well as the serving size. This calculation is based on the information provided by the OpenFoodFacts database for a single bag.
